In [42]:
import zipfile as zipfile
import pandas as pd
import datetime as dt
import os
import unicodedata
import json

print("\nLendo dados das estações pluviometricas...")

for_concat = []
with zipfile.ZipFile('../data/postos.zip', 'r') as t:
    for arq in t.namelist():
        if arq.endswith('.txt'):
            with t.open(arq) as f:
                df = pd.read_csv(f, delimiter=';')
                # Extrair o ID do nome do arquivo e adicionar como uma nova coluna
                df['id'] = arq.split('.')[0]
                for_concat.append(df)

# Exclude empty or all-NA entries before concatenation
for_concat = [
    df for df in for_concat if not df.empty and not df.isna().all().all()
]

registros_chuvas = pd.concat(for_concat)
registros_chuvas.fillna(0, inplace=True)
registros_chuvas.reset_index(drop=True, inplace=True)

# Reordenar as colunas para que 'id' seja a primeira
cols = ['id'] + [col for col in registros_chuvas.columns if col != 'id']
registros_chuvas = registros_chuvas[cols]


Lendo dados das estações pluviometricas...


In [43]:
print("\nFiltrando anos completos...")

#pegar apenas os anos completos
registros_chuvas['Anos'] = registros_chuvas['Anos'].astype(int)
registros_chuvas['Meses'] = registros_chuvas['Meses'].astype(int)
ano = dt.datetime.now().year
Anos_completos = registros_chuvas
Anos_completos = Anos_completos.loc[Anos_completos['Anos'] < ano]


Filtrando anos completos...


In [44]:
print("\nVerificando o primeiro e o ultimo ano de registro de cada posto...")

# Verificar qual o primeiro ano de registro e o ultimo ano de registro de cada posto
primeiro_e_ultimo_ano = Anos_completos.groupby('id')['Anos'].agg(['min', 'max'])
primeiro_e_ultimo_ano.columns = ['Ano_inicial', 'Ano_final']
Anos_completos = Anos_completos.merge(primeiro_e_ultimo_ano, on='id')



Verificando o primeiro e o ultimo ano de registro de cada posto...


In [45]:
# Encontrar qual foi o primeiro mês registrado para cada posto no ano inicial
ano_inicial_df = Anos_completos[Anos_completos['Anos'] == Anos_completos['Ano_inicial']]
primeiro_mes_ano_inicial = ano_inicial_df.groupby('id')['Meses'].min().reset_index()
primeiro_mes_ano_inicial.rename(columns={'Meses': 'Primeiro_mes'}, inplace=True)
Anos_completos = Anos_completos.merge(primeiro_mes_ano_inicial, on='id', how='left')

# Encontrar qual foi o último mês registrado para cada posto no ano final
ano_final_df = Anos_completos[Anos_completos['Anos'] == Anos_completos['Ano_final']]
ultimo_mes_ano_final = ano_final_df.groupby('id')['Meses'].max().reset_index()
ultimo_mes_ano_final.rename(columns={'Meses': 'Ultimo_mes'}, inplace=True)


In [46]:
from datetime import datetime
import pandas as pd
import calendar

def dias_do_mes(ano, mes):
    """Retorna o número correto de dias em um mês, considerando anos bissextos."""
    return calendar.monthrange(ano, mes)[1]

def preencher_anos_faltantes(df):
    postos_unicos = df[['id', 'Postos', 'Municipios']].drop_duplicates()
    anos_faltantes = []
    ano_atual = datetime.now().year

    for _, row in postos_unicos.iterrows():
        posto_id = row['id']
        posto = row['Postos']
        municipio = row['Municipios']

        df_posto = df[(df['id'] == posto_id) & (df['Postos'] == posto) & (df['Municipios'] == municipio)]

        if df_posto['Anos'].notna().any():
            ano_inicial = int(df_posto['Anos'].min())
        else:
            print(f"Aviso: Nenhum ano encontrado para o posto ID {posto_id} - {posto}, município {municipio}. Preenchendo com {ano_atual}.")
            ano_inicial = ano_atual

        anos_completos = set(range(ano_inicial, ano_atual))
        anos_registrados = set(df_posto['Anos'].dropna().unique())
        anos_faltantes_posto = anos_completos - anos_registrados

        for ano in anos_faltantes_posto:
            for mes in range(1, 13):
                dias_no_mes = dias_do_mes(ano, mes)
                falha = {
                    'id': posto_id,
                    'Municipios': municipio,
                    'Postos': posto,
                    'Latitude': df_posto['Latitude'].iloc[0] if not df_posto.empty else 999.0,
                    'Longitude': df_posto['Longitude'].iloc[0] if not df_posto.empty else 999.0,
                    'Anos': ano,
                    'Meses': mes,
                    'Total': 999.0,
                    **{f'Dia{i}': 999.0 if i <= dias_no_mes else 888.0 for i in range(1, 32)}
                }
                anos_faltantes.append(falha)

    df_faltantes = pd.DataFrame(anos_faltantes)
    df = pd.concat([df, df_faltantes], ignore_index=True)
    df.sort_values(by=['id', 'Postos', 'Municipios', 'Anos', 'Meses'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

def preencher_meses_faltantes(df):
    postos_unicos = df[['id', 'Postos', 'Municipios']].drop_duplicates()
    meses_faltantes = []

    for _, row in postos_unicos.iterrows():
        posto_id = row['id']
        posto = row['Postos']
        municipio = row['Municipios']

        df_posto = df[(df['id'] == posto_id) & (df['Postos'] == posto) & (df['Municipios'] == municipio)]
        anos_registrados = df_posto['Anos'].dropna().unique()

        for ano in anos_registrados:
            df_ano = df_posto[df_posto['Anos'] == ano]
            meses_registrados = set(df_ano['Meses'].dropna().unique())
            meses_completos = set(range(1, 13))
            meses_faltantes_ano = meses_completos - meses_registrados

            for mes in meses_faltantes_ano:
                dias_no_mes = dias_do_mes(ano, mes)
                falha = {
                    'id': posto_id,
                    'Municipios': municipio,
                    'Postos': posto,
                    'Latitude': df_posto['Latitude'].iloc[0] if not df_posto.empty else 999.0,
                    'Longitude': df_posto['Longitude'].iloc[0] if not df_posto.empty else 999.0,
                    'Anos': ano,
                    'Meses': mes,
                    'Total': 999.0,
                    **{f'Dia{i}': 999.0 if i <= dias_no_mes else 888.0 for i in range(1, 32)}
                }
                meses_faltantes.append(falha)

    df_faltantes = pd.DataFrame(meses_faltantes)
    df = pd.concat([df, df_faltantes], ignore_index=True)
    df.sort_values(by=['id', 'Postos', 'Municipios', 'Anos', 'Meses'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

# Executar as funções
Anos_completos = preencher_anos_faltantes(Anos_completos)
Anos_completos = preencher_meses_faltantes(Anos_completos)


In [47]:
# Converter colunas numéricas de string para float
numeric_cols = ['Total'] + [f'Dia{i}' for i in range(1, 32)]
for col in numeric_cols:
    Anos_completos[col] = Anos_completos[col].astype(str).str.replace(',', '.').astype(float)

output_dir = "../data/postos_solo"
os.makedirs(output_dir, exist_ok=True)

def remover_acentos(texto):
    return ''.join(c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn')

for posto_id in Anos_completos['id'].unique():
    registros_posto = Anos_completos[Anos_completos['id'] == posto_id]
    nome_posto = registros_posto['Postos'].iloc[0].replace(' ', '_')
    nome_posto = remover_acentos(nome_posto)  # Remover acentos
    nome_arquivo = os.path.join(output_dir, f"{posto_id}_{nome_posto}.csv")
    registros_posto.to_csv(nome_arquivo, index=False, decimal=',')

print("\nArquivos CSV gerados para cada posto na pasta 'postos_solo'.")


Arquivos CSV gerados para cada posto na pasta 'postos_solo'.


In [48]:
print("\nRepreenchendo os novos registros com os meses e anos de primeiro registro")

primeiro_e_ultimo_ano = Anos_completos.groupby('id')['Anos'].agg(['min', 'max'])
primeiro_e_ultimo_ano.columns = ['Ano_inicial', 'Ano_final']
Anos_completos = Anos_completos.merge(primeiro_e_ultimo_ano, on='id')

# Check if the columns exist before attempting to drop them
if 'Ano_inicial_x' in Anos_completos.columns and 'Ano_final_x' in Anos_completos.columns:
    Anos_completos.drop(columns=['Ano_inicial_x', 'Ano_final_x'], inplace=True)

Anos_completos = Anos_completos.merge(primeiro_mes_ano_inicial, on='id', how='left')
Anos_completos = Anos_completos.merge(ultimo_mes_ano_final, on='id', how='left')

# Check if the column exists before attempting to drop it
if 'Primeiro_mes_x' in Anos_completos.columns:
    Anos_completos.drop(columns=['Primeiro_mes_x'], inplace=True)

Anos_completos.rename(columns={
    'Ano_inicial_y': 'Ano_inicial',
    'Ano_final_y': 'Ano_final',
    'Primeiro_mes_y': 'Primeiro_mes'
}, inplace=True)



Repreenchendo os novos registros com os meses e anos de primeiro registro


In [49]:
print("\nVerificando quantos meses de falha existem parada cada posto")

df_sem_extras = Anos_completos.drop(
    columns=['Primeiro_mes', 'Ultimo_mes', 'Ano_inicial', 'Ano_final'])

# Substituir os valores 888.0 por NaN e depois remover essas colunas
df_sem_extras.replace(888.0, pd.NA, inplace=True)
df_sem_extras.dropna(axis=1, how='all', inplace=True)



Verificando quantos meses de falha existem parada cada posto


In [50]:
print("\nVerificando quantos meses de falha existem para cada posto")
dias_cols = [f'Dia{i}' for i in range(1, 32)]

def verificar_meses_falha(df):
    df['Mes_falha'] = df[dias_cols].apply(lambda row: any(day == 999.0 for day in row), axis=1)
    meses_falha_por_posto = df.groupby('id')['Mes_falha'].sum()
    return meses_falha_por_posto

meses_falha_por_posto = verificar_meses_falha(Anos_completos)
Anos_completos = Anos_completos.merge(meses_falha_por_posto.rename('Meses_de_Falha'), on='id', how='left')
Anos_completos = Anos_completos.drop(columns=['Mes_falha'])



Verificando quantos meses de falha existem para cada posto


In [51]:
print(
    "\nCalculando intervalo de dias, meses e anos totais de falhas por posto..."
)

from datetime import datetime

def calcular_intervalo_dias(row):
    ano_inicial = int(row['Ano_inicial'])
    ano_final = dt.datetime.now().year  # Considerar o último ano completo
    intervalo = (ano_final - ano_inicial) * 365

    # Adicionar 1 dia para cada ano bissexto no intervalo
    for year in range(ano_inicial, ano_final + 1):
        if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
            intervalo += 1

    return intervalo

def contar_dias_falha(row):
    return sum(row[dia] == 999.0 for dia in dias_cols if dia != 'Total')

Anos_completos['Ano_inicial'] = Anos_completos['Ano_inicial'].astype(int)
Anos_completos['Ano_final'] = Anos_completos['Ano_final'].astype(int)

Anos_completos['Intervalo_dias'] = Anos_completos.apply(calcular_intervalo_dias, axis=1)
Anos_completos['Dias_de_Falha'] = Anos_completos.apply(contar_dias_falha, axis=1)

total_falhas_por_posto = Anos_completos.groupby('id')['Dias_de_Falha'].sum().reset_index()
total_falhas_por_posto.rename(columns={'Dias_de_Falha': 'Total_Falhas'}, inplace=True)

Anos_completos = Anos_completos.merge(total_falhas_por_posto, on='id', how='left')
Anos_completos['Intervalo_anos'] = (Anos_completos['Ano_final'].astype(int) - Anos_completos['Ano_inicial'].astype(int)) + 1
Anos_completos['Intervalo_meses'] = Anos_completos['Intervalo_anos'] * 12

Anos_completos.drop(columns=['Dias_de_Falha'], inplace=True)
Anos_completos['dias_medidos'] = Anos_completos['Intervalo_dias'] - Anos_completos['Total_Falhas']



Calculando intervalo de dias, meses e anos totais de falhas por posto...


In [55]:
print("\nCalculando a porcentagem de dias de falhas por posto...")

Anos_completos['Porcentagem_dias_Falhas'] = (
    Anos_completos['Total_Falhas'] / Anos_completos['Intervalo_dias']) * 100
Anos_completos['Porcentagem_dias_Falhas'] = Anos_completos[
    'Porcentagem_dias_Falhas'].round(2)


Calculando a porcentagem de dias de falhas por posto...


In [56]:
print("\nVerificando anos de falha por posto...")

# Verificar quantos anos com falha tiveram no intervalo por posto
def contar_anos_falha(df):
    df['Ano_com_falha'] = df[dias_cols].apply(lambda row: any(day == 999.0 for day in row), axis=1)
    anos_falha = df.groupby(['id', 'Anos'])['Ano_com_falha'].any().reset_index()
    anos_falha_por_posto = anos_falha.groupby('id')['Ano_com_falha'].sum().reset_index()
    anos_falha_por_posto.rename(columns={'Ano_com_falha': 'Anos_de_Falha'}, inplace=True)
    return anos_falha_por_posto

anos_falha_por_posto = contar_anos_falha(Anos_completos)
Anos_completos = Anos_completos.merge(anos_falha_por_posto, on='id', how='left')
Anos_completos.drop(columns=['Ano_com_falha'], inplace=True)



Verificando anos de falha por posto...


In [57]:
print("\nCalculando anos completos medidos por posto...")

# Agrupar por id para garantir que os dados são do mesmo posto
Anos_completos['Anos_completos_medidos'] = Anos_completos.groupby('id')['Intervalo_anos'].transform('first') - Anos_completos.groupby('id')['Anos_de_Falha'].transform('first')

Anos_completos['Porcentagem_anos_Falhas'] = (
    Anos_completos['Anos_de_Falha'] / Anos_completos['Intervalo_anos']) * 100
Anos_completos['Porcentagem_anos_Falhas'] = Anos_completos[
    'Porcentagem_anos_Falhas'].round(2)



Calculando anos completos medidos por posto...


In [58]:
print("\nCalculando a média mensal de chuvas por posto...")

def dados_validos(df):
    # Verificar se todos os dias do mês não são 999.0
    df['Mes_valido'] = df[dias_cols].apply(lambda row: all(day != 999.0 for day in row), axis=1)
    
    # Filtrar apenas os meses válidos
    df_validos = df[df['Mes_valido']].copy()
    
    # Remover a coluna auxiliar 'Mes_valido'
    df_validos.drop(columns=['Mes_valido'], inplace=True)
    
    return df_validos

df_dados_validos = dados_validos(Anos_completos)

# Calcular a média mensal de chuvas por posto
ids_unicos = df_dados_validos['id'].unique()

medias_mensais = []

for posto_id in ids_unicos:
    df_posto = df_dados_validos[df_dados_validos['id'] == posto_id]
    
    # Inicializar variáveis para somar os totais e contadores para cada mês
    jan_total = fev_total = mar_total = apr_total = may_total = jun_total = 0
    jul_total = aug_total = sep_total = oct_total = nov_total = dec_total = 0
    
    jan_count = fev_count = mar_count = apr_count = may_count = jun_count = 0
    jul_count = aug_count = sep_count = oct_count = nov_count = dec_count = 0
    
    for _, row in df_posto.iterrows():
        if row['Meses'] == 1:
            jan_total += row['Total']
            jan_count += 1
        elif row['Meses'] == 2:
            fev_total += row['Total']
            fev_count += 1
        elif row['Meses'] == 3:
            mar_total += row['Total']
            mar_count += 1
        elif row['Meses'] == 4:
            apr_total += row['Total']
            apr_count += 1
        elif row['Meses'] == 5:
            may_total += row['Total']
            may_count += 1
        elif row['Meses'] == 6:
            jun_total += row['Total']
            jun_count += 1
        elif row['Meses'] == 7:
            jul_total += row['Total']
            jul_count += 1
        elif row['Meses'] == 8:
            aug_total += row['Total']
            aug_count += 1
        elif row['Meses'] == 9:
            sep_total += row['Total']
            sep_count += 1
        elif row['Meses'] == 10:
            oct_total += row['Total']
            oct_count += 1
        elif row['Meses'] == 11:
            nov_total += row['Total']
            nov_count += 1
        elif row['Meses'] == 12:
            dec_total += row['Total']
            dec_count += 1
    
    medias_mensais.append({
        'id': posto_id,
        'Media_Jan': jan_total / jan_count if jan_count else 999,
        'Media_Fev': fev_total / fev_count if fev_count else 999,
        'Media_Mar': mar_total / mar_count if mar_count else 999,
        'Media_Apr': apr_total / apr_count if apr_count else 999,
        'Media_May': may_total / may_count if may_count else 999,
        'Media_Jun': jun_total / jun_count if jun_count else 999,
        'Media_Jul': jul_total / jul_count if jul_count else 999,
        'Media_Aug': aug_total / aug_count if aug_count else 999,
        'Media_Sep': sep_total / sep_count if sep_count else 999,
        'Media_Oct': oct_total / oct_count if oct_count else 999,
        'Media_Nov': nov_total / nov_count if nov_count else 999,
        'Media_Dec': dec_total / dec_count if dec_count else 999
    })

df_medias_mensais = pd.DataFrame(medias_mensais)
Anos_completos = Anos_completos.merge(df_medias_mensais, on='id', how='left')
# Preencher médias vazias com 999

medias_cols = ['Media_Jan', 'Media_Fev', 'Media_Mar', 'Media_Apr', 'Media_May', 'Media_Jun', 'Media_Jul', 'Media_Aug', 'Media_Sep', 'Media_Oct', 'Media_Nov', 'Media_Dec']
Anos_completos[medias_cols] = Anos_completos[medias_cols].fillna(999)



Calculando a média mensal de chuvas por posto...


In [59]:
print("\nCalculando a média anual de chuvas por posto...")

postos_unicos = Anos_completos['id'].unique()

Anos_completos['Media_Anual'] = 0.0

for posto_id in postos_unicos:
    df_posto = Anos_completos[Anos_completos['id'] == posto_id]
    intervalo_anos = df_posto['Intervalo_anos'].iloc[0]

    # Check if all monthly averages are 999
    if (df_posto[['Media_Jan', 'Media_Fev', 'Media_Mar', 'Media_Apr', 'Media_May', 'Media_Jun', 'Media_Jul', 'Media_Aug', 'Media_Sep', 'Media_Oct', 'Media_Nov', 'Media_Dec']] == 999).all(axis=None):
        media_anual = 999
    else:
        soma_total_chuvas = df_posto[df_posto['Total'] != 999].groupby('Anos')['Total'].sum().sum()
        media_anual = soma_total_chuvas / intervalo_anos

    Anos_completos.loc[Anos_completos['id'] == posto_id, 'Media_Anual'] = media_anual



Calculando a média anual de chuvas por posto...


In [60]:
Anos_completos['Total_meses_intervalo'] = Anos_completos['Intervalo_anos'] * 12

if 'Intervalo_meses' in Anos_completos.columns:
    Anos_completos.drop(columns=['Intervalo_meses'], inplace=True)

Anos_completos['Numero_meses_completos'] = Anos_completos['Total_meses_intervalo'] - Anos_completos['Meses_de_Falha']

def calcular_percentual_meses_falha(df):
    df['Percentual_meses_falha'] = (df['Meses_de_Falha'] / df['Total_meses_intervalo']) * 100
    df['Percentual_meses_falha'] = df['Percentual_meses_falha'].round(2)
    return df

Anos_completos = calcular_percentual_meses_falha(Anos_completos)

In [61]:
colunas_resumo1 = [
    'id','link_csv', 'Postos', 'Municipios', 'Latitude', 'Longitude', 'Ano_inicial', 'Ano_final',
    'Primeiro_mes', 'Ultimo_mes', 'Intervalo_dias', 'dias_medidos', 'Total_Falhas',
    'Porcentagem_dias_Falhas', 'Total_meses_intervalo', 'Numero_meses_completos',
    'Meses_de_Falha', 'Percentual_meses_falha', 'Intervalo_anos', 'Anos_completos_medidos',
    'Anos_de_Falha', 'Porcentagem_anos_Falhas', 'Media_Anual', 'Media_Jan', 'Media_Fev',
    'Media_Mar', 'Media_Apr', 'Media_May', 'Media_Jun', 'Media_Jul', 'Media_Aug',
    'Media_Sep', 'Media_Oct', 'Media_Nov', 'Media_Dec'
    , 'Dia1', 'Dia2', 'Dia3', 'Dia4', 'Dia5', 'Dia6', 'Dia7', 'Dia8', 'Dia9', 'Dia10', 'Dia11', 'Dia12', 'Dia13', 'Dia14', 'Dia15', 'Dia16', 'Dia17', 'Dia18', 'Dia19', 'Dia20', 'Dia21', 'Dia22', 'Dia23', 'Dia24', 'Dia25', 'Dia26', 'Dia27', 'Dia28', 'Dia29', 'Dia30', 'Dia31'
]

database = Anos_completos.copy()

database.drop(columns=['Mes_valido'], inplace=True)

database.rename(columns={
    'id': 'ID',
    'Postos': 'Nome_Posto',
    'Municipios': 'Nome_Municipio',
    'Latitude': 'Coordenada_Y',
    'Longitude': 'Coordenada_X',
    'Ano_inicial': 'Ano_Inicio',
    'Ano_final': 'Ano_Fim',
    'Primeiro_mes': 'Mes_Inicio',
    'Ultimo_mes': 'Mes_Fim',
    'Intervalo_dias': 'Total_dias_intervalo',
    'dias_medidos': 'Dias_dados_medidos',
    'Total_Falhas': 'Dias_falhos',
    'Porcentagem_dias_Falhas': 'Percentual_dias_falhos',
    'Total_meses_intervalo': 'Total_meses_intervalo',
    'Numero_meses_completos': 'Numero_meses_completos',
    'Meses_de_Falha': 'Numero_meses_falha',
    'Percentual_meses_falha': 'Percentual_meses_falha',
    'Intervalo_anos': 'Total_anos_intervalo',
    'Anos_completos_medidos': 'Numero_anos_completos',
    'Anos_de_Falha': 'Numero_anos_falha',
    'Porcentagem_anos_Falhas': 'Percentual_anos_falha',
    'Media_Anual': 'Precipitacao_media_anual',
}, inplace=True)

# Arredondar as colunas selecionadas para 2 casas decimais
colunas_para_arredondar = [col for col in database.columns if col not in ['Coordenada_X', 'Coordenada_Y']]
database[colunas_para_arredondar] = database[colunas_para_arredondar].round(2)
database['Coordenada_X'] = database['Coordenada_X'].round(8)
database['Coordenada_Y'] = database['Coordenada_Y'].round(8)

# Geração do DATAFRAME que será inserido no banco de dados
database.to_csv('../data/maindatabase.csv', index=False, decimal='.')

In [65]:
print ("Geração do dataframe usado na tabela no site.") 

#print(Anos_completos.columns)

colunas_resumo = [
    'id','link_csv', 'Postos', 'Municipios', 'Latitude', 'Longitude', 'Ano_inicial', 'Ano_final',
    'Primeiro_mes', 'Ultimo_mes', 'Intervalo_dias', 'dias_medidos', 'Total_Falhas',
    'Porcentagem_dias_Falhas', 'Total_meses_intervalo', 'Numero_meses_completos',
    'Meses_de_Falha', 'Percentual_meses_falha', 'Intervalo_anos', 'Anos_completos_medidos',
    'Anos_de_Falha', 'Porcentagem_anos_Falhas', 'Media_Anual', 'Media_Jan', 'Media_Fev',
    'Media_Mar', 'Media_Apr', 'Media_May', 'Media_Jun', 'Media_Jul', 'Media_Aug',
    'Media_Sep', 'Media_Oct', 'Media_Nov', 'Media_Dec'
]

links_df = pd.read_csv('../data/links.csv', header=None, names=['id', 'link_csv'], encoding='latin1')

Anos_completos['id'] = Anos_completos['id'].astype(str)

links_df['id'] = links_df['id'].astype(str)

Anos_completos = Anos_completos.merge(links_df, on='id', how='left')



Geração do dataframe usado na tabela no site.


In [67]:
df_resumo = Anos_completos[colunas_resumo].copy()

df_resumo = df_resumo.groupby('id').first().reset_index()

# Renomear as colunas conforme o formato desejado
df_resumo.rename(columns={
    'id': 'ID',
    'Postos': 'Nome_Posto',
    'Municipios': 'Nome_Municipio',
    'Latitude': 'Coordenada_Y',
    'Longitude': 'Coordenada_X',
    'Ano_inicial': 'Ano_Inicio',
    'Ano_final': 'Ano_Fim',
    'Primeiro_mes': 'Mes_Inicio',
    'Ultimo_mes': 'Mes_Fim',
    'Intervalo_dias': 'Total_dias_intervalo',
    'dias_medidos': 'Dias_dados_medidos',
    'Total_Falhas': 'Dias_falhos',
    'Porcentagem_dias_Falhas': 'Percentual_dias_falhos',
    'Total_meses_intervalo': 'Total_meses_intervalo',
    'Numero_meses_completos': 'Numero_meses_completos',
    'Meses_de_Falha': 'Numero_meses_falha',
    'Percentual_meses_falha': 'Percentual_meses_falha',
    'Intervalo_anos': 'Total_anos_intervalo',
    'Anos_completos_medidos': 'Numero_anos_completos',
    'Anos_de_Falha': 'Numero_anos_falha',
    'Porcentagem_anos_Falhas': 'Percentual_anos_falha',
    'Media_Anual': 'Precipitacao_media_anual',
    'Media_Jan': 'Mes_Jan',
    'Media_Fev': 'Mes_Fev',
    'Media_Mar': 'Mes_Mar',
    'Media_Apr': 'Mes_Apr',
    'Media_May': 'Mes_May',
    'Media_Jun': 'Mes_Jun',
    'Media_Jul': 'Mes_Jul',
    'Media_Aug': 'Mes_Aug',
    'Media_Sep': 'Mes_Sep',
    'Media_Oct': 'Mes_Oct',
    'Media_Nov': 'Mes_Nov',
    'Media_Dec': 'Mes_Dec'
}, inplace=True)

# Converter a coluna 'ID' para numérica para evitar problemas de ordenação
df_resumo['ID'] = pd.to_numeric(df_resumo['ID'], errors='coerce')

# Remover linhas com valores NaN na coluna 'ID' (se houver)
df_resumo.dropna(subset=['ID'], inplace=True)

# Ordenar com base na coluna 'ID'
df_resumo.sort_values(by='ID', inplace=True)

# Arredondar todas as médias e percentuais para 2 casas decimais
colunas_arredondar = [
    'Precipitacao_media_anual', 'Mes_Jan', 'Mes_Fev', 'Mes_Mar', 'Mes_Apr', 'Mes_May',
    'Mes_Jun', 'Mes_Jul', 'Mes_Aug', 'Mes_Sep', 'Mes_Oct', 'Mes_Nov', 'Mes_Dec',
    'Percentual_dias_falhos', 'Percentual_meses_falha', 'Percentual_anos_falha'
]
df_resumo[colunas_arredondar] = df_resumo[colunas_arredondar].round(2)

# Salvar o DataFrame em um arquivo CSV
df_resumo.to_csv('../data/resumo_postos_individual.csv', index=False, decimal=',')

# Diagnóstico para verificar duplicados ou valores nulos na coluna 'ID'
duplicados = df_resumo['ID'].duplicated().sum()
valores_nulos = df_resumo['ID'].isnull().sum()


In [ ]:
# Converter todos os valores numéricos para strings
df_resumo = df_resumo.map(
    lambda x: str(x) if isinstance(x, (int, float)) else x)

def remover_acentos(texto):
    return ''.join(c for c in unicodedata.normalize('NFD', texto)
                   if unicodedata.category(c) != 'Mn')

df_resumo = df_resumo.map(lambda x: remover_acentos(x)
                                              if isinstance(x, str) else x)

# Converter o dataframe para uma lista de dicionários
dados_formatados_resumo = df_resumo.to_dict(orient='records')

# Ler o arquivo links.csv com encoding 'latin1'
links_df = pd.read_csv('../data/links.csv', header=None, names=['link_csv'], encoding='latin1')

# Adicionar a coluna link_csv ao dataframe df_resumo
df_resumo['link_csv'] = links_df['link_csv'].values

colunas = list(df_resumo.columns)  # Obtém a lista de colunas
colunas.insert(1, colunas.pop(colunas.index('link_csv')))  # Move 'link_csv' para a segunda posição
df_resumo = df_resumo[colunas]  # Reorganiza o DataFrame com a nova ordem de colunas



In [ ]:
with open('../data/dados_formatados_resumo.json', 'w', encoding='utf-8') as f:
    json.dump(dados_formatados_resumo, f, ensure_ascii=False, indent=4)


print("Arquivo JSON gerado com sucesso.")

Arquivo JSON gerado com sucesso.


In [ ]:
df_ibge = pd.read_csv('../data/municipios.csv', encoding='latin1', sep=';')
df = pd.read_csv('../data/maindatabase.csv', encoding='latin1', sep=',')

print(df_ibge.head())
print(df_ibge.columns)

# Criar dicionário para a tabela de municípios
municipios = df_ibge[['cod_ibge', 'municipio']].drop_duplicates().reset_index(drop=True)
municipios.insert(0, 'id', range(len(municipios)))

postos = df[['ID', 'Nome_Posto', 'Dias_dados_medidos', 'Dias_falhos', 
             'Numero_meses_completos', 'Numero_meses_falha', 'Numero_anos_falha', 
             'Numero_anos_completos', 'Precipitacao_media_anual', 'Coordenada_Y', 'Coordenada_X']].drop_duplicates()

integer_columns = ['ID', 'Dias_dados_medidos', 'Dias_falhos', 'Numero_meses_completos', 
                   'Numero_meses_falha', 'Numero_anos_falha', 'Numero_anos_completos']
postos[integer_columns] = postos[integer_columns].astype(int)

postos.rename(columns={'ID': 'id_posto'}, inplace=True)

postos['coordenadas'] = postos['Coordenada_Y'].astype(str) + ',' + postos['Coordenada_X'].astype(str)
postos.drop(columns=['Coordenada_Y', 'Coordenada_X'], inplace=True)

registros = df[['Dia1', 'Total', 'Meses', 'Anos']].rename(columns={
    'Dia1': 'dia',
    'Total': 'total_dia',
    'Meses': 'mes',
    'Anos': 'ano'
})

registros.insert(0, 'id', range(len(registros)))

municipios.columns = municipios.columns.str.lower()
postos.columns = postos.columns.str.lower()
registros.columns = registros.columns.str.lower()

# Procurar o registro no DataFrame 'postos' onde a coluna 'precipitacao_media_anual' é igual a 930.39
registro = postos['precipitacao_media_anual']

# Exibir o registro encontrado
# Substituir pontos por vírgulas nas colunas de tipo float
postos['precipitacao_media_anual'] = postos['precipitacao_media_anual'].apply(lambda x: str(x).replace('.', ','))

# Exibir o registro encontrado
print(postos['precipitacao_media_anual'])


  municipio  cod_ibge
0   Abaiara   2300101
1   Acarape   2300150
2    Acaraú   2300200
3  Acopiara   2300309
4    Aiuaba   2300408
Index(['municipio', 'cod_ibge'], dtype='object')
0          930.39
528        773.36
1044       810.32
1656       735.40
2268       759.44
           ...   
285900    1021.74
286428     846.06
286920     998.16
287532    1119.77
288144     877.39
Name: precipitacao_media_anual, Length: 827, dtype: float64


In [ ]:
import csv
import unicodedata
import re

def limpar_caracteres_especiais(texto):
    """
    Remove acentos e outros caracteres especiais de uma string.

    Args:
        texto: A string a ser limpa.

    Returns:
        A string com acentos e caracteres especiais removidos.
    """
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
    texto = re.sub(r'[^a-zA-Z0-9\s]', '', texto)
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

# Caminho para o arquivo CSV
caminho_arquivo = "../data/maindatabase.csv"
nome_arquivo_saida = "seu_arquivo_limpo.csv"
coluna_alvo = "Nome_Municipio"
delimitador = ";"

try:
    with open(caminho_arquivo, 'r', newline='', encoding='latin-1') as arquivo_csv:
        leitor_csv = csv.reader(arquivo_csv, delimiter=delimitador)
        # ... o restante do seu código ...
        cabecalho = next(leitor_csv)  # Lê o cabeçalho
        linhas_limpas = [cabecalho]  # Inclui o cabeçalho no novo arquivo

        try:
            indice_municipio = cabecalho.index(coluna_alvo)
        except ValueError:
            print(f"A coluna '{coluna_alvo}' não foi encontrada no cabeçalho.")
            exit()

        # Lê e processa as linhas do arquivo
        for linha in leitor_csv:
            linha_limpa = list(linha)  # Cria uma cópia mutável da linha
            if indice_municipio < len(linha_limpa):
                linha_limpa[indice_municipio] = limpar_caracteres_especiais(linha[indice_municipio])
            linhas_limpas.append(linha_limpa)

    # Escreve as linhas limpas em um novo arquivo CSV
    with open(nome_arquivo_saida, 'w', newline='', encoding='utf-8') as arquivo_saida:
        escritor_csv = csv.writer(arquivo_saida, delimiter=delimitador)
        escritor_csv.writerows(linhas_limpas)

    print(f"Arquivo '{nome_arquivo_saida}' limpo com sucesso!")

except FileNotFoundError:
    print(f"Arquivo não encontrado: {caminho_arquivo}")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

A coluna 'Nome_Municipio' não foi encontrada no cabeçalho.
Arquivo 'seu_arquivo_limpo.csv' limpo com sucesso!


: 

In [87]:
import pandas as pd

# Carregar dados
df_ibge = pd.read_csv('../data/municipios.csv', encoding='latin1', sep=';')

df = pd.read_csv('../data/maindatabase.csv', encoding='latin1', sep=',')

# Processar municípios
municipios = df_ibge[['cod_ibge', 'municipio']].drop_duplicates().reset_index(drop=True)
municipios.insert(0, 'id', range(len(municipios)))

# Padronizar nomes para minúsculas e remover espaços extras
municipios['municipio'] = municipios['municipio'].str.strip().str.lower()

# Processar postos
postos = df[['ID', 'Nome_Posto', 'Nome_Municipio', 'Dias_dados_medidos', 'Dias_falhos', 
             'Numero_meses_completos', 'Numero_meses_falha', 'Numero_anos_falha', 
             'Numero_anos_completos', 'Precipitacao_media_anual', 'Coordenada_Y', 'Coordenada_X']].drop_duplicates()

postos['Nome_Municipio'] = postos['Nome_Municipio'].str.strip().str.lower()
postos['Nome_Posto'] = postos['Nome_Posto'].str.strip().str.lower()

postos = postos.merge(municipios[['cod_ibge', 'municipio']], 
                      left_on='Nome_Municipio', 
                      right_on='municipio', 
                      how='left').drop(columns=['municipio'])



In [88]:
postos

,ID,Nome_Posto,Nome_Municipio,Dias_dados_medidos,Dias_falhos,Numero_meses_completos,Numero_meses_falha,Numero_anos_falha,Numero_anos_completos,Precipitacao_media_anual,Coordenada_Y,Coordenada_X,cod_ibge
0,1,abaiara,abaiara,16048,34,526,2,2,42,930.39,-7.361528,-39.035500,2300101.0
1,10,apuiares,apuiarã©s,15683,34,512,4,3,40,773.36,-3.950000,-39.433306,NaN
2,100,nova russas,nova russas,18569,72,607,5,3,48,810.32,-4.699528,-40.568472,2309300.0
3,101,novo oriente,novo oriente,18616,25,609,3,2,49,735.40,-5.540250,-40.770417,2309409.0
4,102,oros,orã³s,16903,276,555,9,3,44,759.44,-6.243778,-38.912111,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
822,95,moraujo,moraãºjo,13365,2717,436,92,12,32,1021.74,-3.466917,-40.677917,NaN
823,96,morrinhos,morrinhos,14865,122,487,5,3,38,846.06,-3.232250,-40.129222,2308906.0
824,97,mucambo,mucambo,18408,233,601,11,6,45,998.16,-3.905694,-40.743306,2309003.0
825,98,mulungu,mulungu,18560,81,607,5,4,47,1119.77,-4.305722,-38.997194,2309102.0


In [ ]:
batch_size = 5000

def insert_batch(table_name, data):
    """ Insere os dados em lotes para evitar sobrecarga """
    test_data = data.to_dict(orient='records')
    for i in range(0, len(test_data), batch_size):
        batch = test_data[i:i + batch_size]
        supabase.table(table_name).insert(batch).execute()
        print(f"Inserindo lote na tabela {table_name}... Parte {i//batch_size + 1}")

# Enviar dados para o Supabase
insert_batch("municipio", municipios)
insert_batch("posto", postos)
insert_batch("registro", registros)

print("Inserção concluída com sucesso!")

,id,cod_ibge,municipio,id_posto
0,0,2300101,abaiara,NaN
1,1,2300150,acarape,NaN
2,2,2300200,acaraú,NaN
3,3,2300309,acopiara,NaN
4,4,2300408,aiuaba,NaN
...,...,...,...,...
127,127,2309706,pacatuba,NaN
128,128,2309805,pacoti,NaN
129,129,2309904,pacujá,NaN
130,130,2310001,palhano,NaN
